In [1]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly.subplots import make_subplots

/Users/macbook/anaconda3/lib/python3.7/site-packages/google/colab/data_table.py:30: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



#### Getting data

In [2]:
demo = pd.read_csv('/Users/macbook/Downloads/Sumeet_Raw_Data/Demographics-Table 1.csv', sep=';')
demo = demo.loc[:,:'BUSINESS_LINE']
engage = pd.read_csv('/Users/macbook/Downloads/Sumeet_Raw_Data/Engagement-Table 1.csv', sep=';')
engage = engage.loc[:,:'How satisfied or dissatisfied are you working here?']
perform = pd.read_csv('/Users/macbook/Downloads/Sumeet_Raw_Data/Performance-Table 1.csv', sep=';')
perform = perform.loc[:,:'PERFORMANCE_RATING']

#### Merging tables to the final dataframe

In [3]:
df = pd.merge(demo, perform, on='EMPLOYEE_ID', how='left')
df = pd.merge(df, engage, on='EMPLOYEE_ID', how='left')
cols = ['PERFORMANCE_RATING',
       'Would you work with your manager again in the future?',
       'What percentage of time do you spend doing the work you enjoy?',
       'How many more years do you plan to stay?',
       'To what extent are you gaining the skills and experiences you need to develop your career?',
       'How satisfied or dissatisfied are you working here?']

df = df.dropna(subset=cols)
df.shape

(485, 13)

## Analysis objectives
+ What is the relationship between performance ratings and Work You Enjoy (WYE)?
+ Are there any differences in Ratings or Engagement by Gender? If so, are the differences meaningful and/or significant?
+ How are our technical employees doing inside and outside of the Tech business line?
+ How might you explain lower scores on WYE?
+ Based on your findings, what action(s) would you take and how would you track progress over time?!


### Functions to visualize data

In [4]:
# Plot to get a distribution of the target variable

def dist_plot(target, title_x, nbins, nticks_x=12):
    
    titles = ['<b>Technical: Yes</b>','<b>Employee Type: Individual Contributor</b>',
              '<b>Business Line: Tech</b>','<b>Technical: No</b>',
              '<b>Employee Type: Manager</b>','<b>Business Line: Non-Tech</b>']
    colors = ['orange', 'slateblue']
    
    fig = make_subplots(rows=2, cols=3, vertical_spacing = 0.1, horizontal_spacing = 0.12, 
                        subplot_titles=(titles))
    
    
    ## Technical
    # Gender & Technical & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['TECHNICAL']=='Yes')][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, name=gender), 1,1)
  
    # Gender & Non-Technical & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['TECHNICAL']=='No')][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, name=gender, showlegend=False), 2,1)
     
    ## Employee Type
    # Gender & 'Individual Contributor' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['EMPLOYEE_TYPE']=='Individual Contributor')][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, name=gender, showlegend=False), 1,2)    
    
    # Gender & 'Manager' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['EMPLOYEE_TYPE']=='Manager')][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, name=gender, showlegend=False), 2,2)
    
    
    ## Business line
    # Gender & 'Tech' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['BUSINESS_LINE']=='Tech')][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, name=gender, showlegend=False), 1,3)    
    
    # Gender & 'Non-tech' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['BUSINESS_LINE']!='Tech')][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, name=gender, showlegend=False), 2,3)
        
    # Layout customization
    axis_y = dict(nticks=5, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11, ticksuffix='%')
    axis_x = dict(nticks=nticks_x, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11)

    fig.update_layout(barmode='overlay', width=900, height=550, plot_bgcolor='rgba(0,0,0,0)',
                     margin=dict(l=0, r=0, t=50, b=0), showlegend=True, bargap=0.1, 
                     yaxis = axis_y, yaxis2 = axis_y, yaxis3 = axis_y, yaxis4 = axis_y, yaxis5 = axis_y, yaxis6 = axis_y,
                     xaxis = axis_x, xaxis2 = axis_x, xaxis3 = axis_x, xaxis4 = axis_x, xaxis5 = axis_x, xaxis6 = axis_x,
                     )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=11)
    
    for element in fig.layout:
        if 'yaxis' in element and any([x in element[-1] for x in ['s','4']]):
            fig.layout[element]['title_text'] = 'Percentage of employees'
            
        if 'xaxis' in element and any([x in element[-1] for x in ['4','5', '6']]):
            fig.layout[element]['title_text'] = title_x
        
    fig.show()
    
    return fig 

In [5]:
# Bar plot to get absolute values of the target variable

def bar_plot(target, title_y):
    titles = ['<b>Technical: Yes</b>','<b>Employee Type: <br> Individual Contributor</b>',
              '<b>Business Line: <br> Tech</b>','<b>Technical: No</b>',
              '<b>Employee Type: <br> Manager</b>','<b>Business Line: <br> Non-Tech</b>']
    colors = ['orange', 'slateblue']
    
    fig = make_subplots(rows=2, cols=3, vertical_spacing = 0.1, horizontal_spacing = 0.12, 
                        subplot_titles=(titles))
    
    
    ## Technical
    # Gender & Technical & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & (df['TECHNICAL']=='Yes')][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[gender], marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False),1,1)

    # Gender & Non-Technical & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['TECHNICAL']=='No')][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[gender], marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False),2,1)
    
    
    ## Employee Type
    # Gender & 'Individual Contributor' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['EMPLOYEE_TYPE']=='Individual Contributor')][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[gender], marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False), 1,2)
    
    # Gender & 'Manager' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['EMPLOYEE_TYPE']=='Manager')][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[gender], marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False), 2,2)
    
    
    ## Business line
    # Gender & 'Tech' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['BUSINESS_LINE']=='Tech')][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[gender], marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False), 1,3)    
    
    # Gender & 'Non-tech' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['BUSINESS_LINE']!='Tech')][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[gender], marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False), 2,3)
        
    # Layout customization
    axis_y = dict(nticks=10, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11)
    axis_x = dict(nticks=10, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11)

    fig.update_layout(barmode='overlay', width=450, height=550, plot_bgcolor='rgba(0,0,0,0)',
                     margin=dict(l=0, r=0, t=50, b=0), showlegend=False, bargap=0.1, 
                     yaxis = axis_y, yaxis2 = axis_y, yaxis3 = axis_y, yaxis4 = axis_y, yaxis5 = axis_y, yaxis6 = axis_y,
                     xaxis = axis_x, xaxis2 = axis_x, xaxis3 = axis_x, xaxis4 = axis_x, xaxis5 = axis_x, xaxis6 = axis_x,
                     )
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=11)
    
    for element in fig.layout:
        if 'yaxis' in element and any([x in element[-1] for x in ['s','4']]):
            fig.layout[element]['title_text'] = title_y

    fig.show()
    
    return fig 

### Performance

In [6]:
fig = dist_plot('PERFORMANCE_RATING','Performance Rate', nbins=10)

In [7]:
fig = bar_plot('PERFORMANCE_RATING','Performance')

### Work You Enjoy

In [8]:
col = 'What percentage of time do you spend doing the work you enjoy?'
min_percent = df[col].str.replace('%', '').str.split('-', expand=True)[0].astype(float)
max_percent = df[col].str.replace('%', '').str.split('-', expand=True)[1].astype(float)
df['WYE'] = (min_percent + max_percent)/2

In [9]:
fig = dist_plot('WYE','WYE', nbins=10)

In [10]:
fig = bar_plot('WYE','WYE')

### Would you work with your manager again in the future?

In [11]:
col = 'Would you work with your manager again in the future?'
df['WorkManagerAgain'] = df[col].str.replace('Very Favorable','2').str.replace('Favorable','1').str.replace('Neutral','0')
df['WorkManagerAgain'] = df['WorkManagerAgain'].str.replace('Very Unfavorable','-2').str.replace('Unfavorable','-1').str.replace('Prefer not to answer','0')
df['WorkManagerAgain'] = df['WorkManagerAgain'].astype(float)
fig = dist_plot('WorkManagerAgain','Work with Manager Again', nbins=5, nticks_x=8)

In [12]:
fig = bar_plot('WorkManagerAgain', title_y='Work with Manager Again')

### How many more years do you plan to stay?

In [13]:
col = 'How many more years do you plan to stay?'
df['YearsStay'] = df[col].str.replace('<1 Year','0.5').str.replace('>10 Years','10').str.replace('3-5 Years','4').str.replace('1-3 Years','2').str.replace('5-10 Years','7.5')
df['YearsStay'] = df['YearsStay'].astype(float)

fig = dist_plot('YearsStay','Planning Years to Stay', nbins=6)

In [14]:
fig = bar_plot('YearsStay','Planning Years to Stay')

### To what extent are you gaining the skills and experiences you need to develop your career?

In [15]:
col = 'To what extent are you gaining the skills and experiences you need to develop your career?'
df['SkillsNeedCarreer'] = df[col].str.replace('Very Favorable','2').fillna('-999').str.replace('Favorable','1').str.replace('Neutral','0')
df['SkillsNeedCarreer'] = df['SkillsNeedCarreer'].str.replace('Very Unfavorable','-2').str.replace('Unfavorable','-1')
df['SkillsNeedCarreer'] = df['SkillsNeedCarreer'].astype(float)

fig = dist_plot('SkillsNeedCarreer','Skills Needeed for Carreer', nbins=6, nticks_x=8)

In [16]:
fig = bar_plot('SkillsNeedCarreer', title_y='Skills Needed for Carreer')

### How satisfied or dissatisfied are you working here?

In [17]:
col = 'How satisfied or dissatisfied are you working here?'
df['SatisfiedWork'] = df[col].str.replace('Very Favorable','2').fillna('-999').str.replace('Favorable','1').str.replace('Neutral','0')
df['SatisfiedWork'] = df['SatisfiedWork'].str.replace('Very Unfavorable','-2').str.replace('Unfavorable','-1')
df['SatisfiedWork'] = df['SatisfiedWork'].astype(float)

fig = dist_plot('SatisfiedWork','Satisfied Working', nbins=6, nticks_x=8)

In [18]:
fig = bar_plot('SatisfiedWork', title_y='Satisfied Working')

## How are our technical employees doing inside and outside of the Tech business line?

In [19]:
def dist_plot_tech(target, title_x, nbins, nticks_x=12):

    titles = ['<b>Gender: Male</b>','<b>Employee Type: Individual Contributor</b>',
             '<b>Gender: Female</b>','<b>Employee Type: Manager</b>']
    colors = ['red', 'blue']
    
    fig = make_subplots(rows=2, cols=2, vertical_spacing = 0.1, horizontal_spacing = 0.12, 
                            subplot_titles=(titles))
    

    ## Gender
    names = ['Tech Line','Non-tech Line']
    for i, line in enumerate([['Tech'], ['Sales', 'Business']]):
        data = df[(df['GENDER']=='Male') & 
                  (df['TECHNICAL']=='Yes') & 
                  (df['BUSINESS_LINE'].isin(line))][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, name=names[i]), 1,1)

    names = ['Tech Line','Non-tech Line']
    for i, line in enumerate([['Tech'], ['Sales', 'Business']]):
        data = df[(df['GENDER']=='Female') & 
                  (df['TECHNICAL']=='Yes') & 
                  (df['BUSINESS_LINE'].isin(line))][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, showlegend=False), 2,1)


    ## Employee type
    for i, line in enumerate([['Tech'], ['Sales', 'Business']]):
        data = df[(df['EMPLOYEE_TYPE']=='Individual Contributor') & 
                  (df['TECHNICAL']=='Yes') & 
                  (df['BUSINESS_LINE'].isin(line))][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, showlegend=False), 1,2)    

    for i, line in enumerate([['Tech'], ['Sales', 'Business']]):
        data = df[(df['EMPLOYEE_TYPE']=='Manager') & 
                  (df['TECHNICAL']=='Yes') & 
                  (df['BUSINESS_LINE'].isin(line))][target]
        fig.add_trace(go.Histogram(x=data, histnorm='percent', nbinsx=nbins, 
                                   marker_color=colors[i], opacity=0.8, showlegend=False), 2,2) 


    # Layout customization
    axis_y = dict(nticks=5, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11, ticksuffix='%')
    axis_x = dict(nticks=nticks_x, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11)

    fig.update_layout(barmode='overlay', width=900, height=550, plot_bgcolor='rgba(0,0,0,0)',
                     margin=dict(l=0, r=0, t=50, b=0), showlegend=True, bargap=0.1,  
                     yaxis = axis_y, yaxis2 = axis_y, yaxis3 = axis_y, yaxis4 = axis_y,
                     xaxis = axis_x, xaxis2 = axis_x, xaxis3 = axis_x, xaxis4 = axis_x,
                     )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=11)

    for element in fig.layout:
        if 'yaxis' in element and any([x in element[-1] for x in ['s','3']]):
            fig.layout[element]['title_text'] = 'Percentage of employees'

        if 'xaxis' in element and any([x in element[-1] for x in ['3','4']]):
            fig.layout[element]['title_text'] = title_x

    fig.show()
    return fig

In [20]:
fig = dist_plot_tech('PERFORMANCE_RATING', title_x='Performance', nbins=8)

In [21]:
fig = dist_plot_tech('WYE', title_x='WYE', nbins=10)

In [22]:
def bar_plot_tech(target, title_y):

    titles = ['<b>Gender: <br> Male</b>','<b>Employee Type: <br>Individual <br>Contributor</b>',
             '<b>Gender: <br> Female</b>','<b>Employee Type: <br>Manager</b>']

    fig = make_subplots(rows=2, cols=2, vertical_spacing = 0.1, horizontal_spacing = 0.12, 
                            subplot_titles=(titles))
    colors = ['red', 'blue']

    ## Gender
    names = ['Tech Line','Non-tech Line']
    for i, line in enumerate([['Tech'], ['Sales', 'Business']]):
        data = df[(df['GENDER']=='Male') & 
                  (df['TECHNICAL']=='Yes') & 
                  (df['BUSINESS_LINE'].isin(line))][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[names[i]], marker_color=colors[i], opacity=0.8), 1,1)    


    for i, line in enumerate([['Tech'], ['Sales', 'Business']]):
        data = df[(df['GENDER']=='Female') & 
                  (df['TECHNICAL']=='Yes') & 
                  (df['BUSINESS_LINE'].isin(line))][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[names[i]], marker_color=colors[i], opacity=0.8), 2,1)    


    ## Employee type
    for i, line in enumerate([['Tech'], ['Sales', 'Business']]):
        data = df[(df['EMPLOYEE_TYPE']=='Individual Contributor') & 
                  (df['TECHNICAL']=='Yes') & 
                  (df['BUSINESS_LINE'].isin(line))][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[names[i]], marker_color=colors[i], opacity=0.8), 1,2)        

    for i, line in enumerate([['Tech'], ['Sales', 'Business']]):
        data = df[(df['EMPLOYEE_TYPE']=='Manager') & 
                  (df['TECHNICAL']=='Yes') & 
                  (df['BUSINESS_LINE'].isin(line))][target].mean()
        fig.add_trace(go.Bar(y=[data], x=[names[i]], marker_color=colors[i], opacity=0.8), 2,2)     


    # Layout customization
    axis_y = dict(nticks=10, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11)
    axis_x = dict(showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11, tickfont_color='white')

    fig.update_layout(barmode='group', width=350, height=550, plot_bgcolor='rgba(0,0,0,0)',
                      
                     margin=dict(l=0, r=0, t=50, b=0), showlegend=False, bargap=0.1, 
                     yaxis = axis_y, yaxis2 = axis_y, yaxis3 = axis_y, yaxis4 = axis_y,
                     xaxis = axis_x, xaxis2 = axis_x, xaxis3 = axis_x, xaxis4 = axis_x,
                     )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=11)

    for element in fig.layout:
        if 'yaxis' in element and any([x in element[-1] for x in ['s','3']]):
            fig.layout[element]['title_text'] = title_y

        if 'xaxis' in element and any([x in element[-1] for x in ['3','4']]):
            fig.layout[element]['title_text'] = ''

    fig.show()
    
    return fig

In [23]:
fig = bar_plot_tech('PERFORMANCE_RATING','Performance')

In [24]:
fig = bar_plot_tech('WYE','WYE')

## What is the relationship between performance ratings and Work You Enjoy (WYE)?

In [25]:
def line_plot(target, title_x):
    fig = go.Figure()

    data = df[df['TECHNICAL']=='Yes']
    plot_df = data.groupby([target])['WYE'].mean().round(2).reset_index()
    fig.add_trace(go.Scatter(y=plot_df['WYE'], x=plot_df[target], name='Technical'))

    data = df[df['TECHNICAL']=='No']
    plot_df = data.groupby([target])['WYE'].mean().round(2).reset_index()
    fig.add_trace(go.Scatter(y=plot_df['WYE'], x=plot_df[target], name='Non-Technical'))

    data = df[df['GENDER']=='Male']
    plot_df = data.groupby([target])['WYE'].mean().round(2).reset_index()
    fig.add_trace(go.Scatter(y=plot_df['WYE'], x=plot_df[target], name='Male'))

    data = df[df['GENDER']=='Female']
    plot_df = data.groupby([target])['WYE'].mean().round(2).reset_index()
    fig.add_trace(go.Scatter(y=plot_df['WYE'],  x=plot_df[target], name='Female'))

    data = df[df['EMPLOYEE_TYPE']=='Individual Contributor']
    plot_df = data.groupby([target])['WYE'].mean().round(2).reset_index()
    fig.add_trace(go.Scatter(y=plot_df['WYE'], x=plot_df[target],name='Individual Contributor'))

    data = df[df['EMPLOYEE_TYPE']=='Manager']
    plot_df = data.groupby([target])['WYE'].mean().round(2).reset_index()
    fig.add_trace(go.Scatter(y=plot_df['WYE'], x=plot_df[target], name='Manager'))


    axis_y = dict(nticks=10, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', title_text='WYE', titlefont_size=11)
    axis_x = dict(nticks=8, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', title_text=title_x, titlefont_size=11)

    fig.update_layout(width=600, height=350, plot_bgcolor='rgba(0,0,0,0)',
                     margin=dict(l=0, r=0, t=50, b=0), showlegend=True,
                     bargap=0.1, barmode='group', yaxis = axis_y, xaxis = axis_x)
    fig.show()
    return fig

In [26]:
fig = line_plot('PERFORMANCE_RATING', 'Performance')

In [27]:
fig = line_plot('LEVEL', 'Level')

## How might you explain lower scores on WYE?

In [28]:
def wye_plot(target, title_y):
    titles = ['<b>Technical: Yes</b>','<b>Employee Type: Individual Contributor</b>',
              '<b>Business Line: Tech</b>','<b>Technical: No</b>',
              '<b>Employee Type: Manager</b>','<b>Business Line: Non-Tech</b>']
    colors = ['orange', 'slateblue']
    
    fig = make_subplots(rows=2, cols=3, vertical_spacing = 0.1, horizontal_spacing = 0.12, 
                        subplot_titles=(titles))
    
    
    ## Technical
    # Gender & Technical & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['TECHNICAL']=='Yes')].groupby('WYE')[target].mean()
        
        fig.add_trace(go.Bar(x=data.index, y=data, marker_color=colors[i], opacity=0.8, name=gender),1,1)

    # Gender & Non-Technical & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['TECHNICAL']=='No')].groupby('WYE')[target].mean()
        
        fig.add_trace(go.Bar(x=data.index, y=data, marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False),2,1)
    
    
    ## Employee type
    # Gender & 'Individual Contributor' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['EMPLOYEE_TYPE']=='Individual Contributor')].groupby('WYE')[target].mean()
        
        fig.add_trace(go.Bar(x=data.index, y=data, marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False), 1,2)
    
    # Gender & 'Manager' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['EMPLOYEE_TYPE']=='Manager')].groupby('WYE')[target].mean()
        
        fig.add_trace(go.Bar(x=data.index, y=data, marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False), 2,2)
    
    
    ## Business line
    # Gender & 'Tech' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['BUSINESS_LINE']=='Tech')].groupby('WYE')[target].mean()
        
        fig.add_trace(go.Bar(x=data.index, y=data, marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False), 1,3)    
    
    # Gender & 'Non-tech' & Targer Variable
    for i, gender in enumerate(['Male', 'Female']):
        data = df[(df['GENDER']==gender) & 
                  (df['BUSINESS_LINE']!='Tech')].groupby('WYE')[target].mean()
        
        fig.add_trace(go.Bar(x=data.index, y=data, marker_color=colors[i], opacity=0.8, 
                             name=gender, showlegend=False), 2,3)
        
    # Layout customization
    axis_y = dict(nticks=5, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11)
    axis_x = dict(nticks=10, showgrid=True, gridwidth=0.1, gridcolor='#EEEEEE', titlefont_size=11)

    fig.update_layout(barmode='overlay', width=900, height=550, plot_bgcolor='rgba(0,0,0,0)',
                     margin=dict(l=0, r=0, t=50, b=0), showlegend=True, bargap=0.1, 
                     yaxis = axis_y, yaxis2 = axis_y, yaxis3 = axis_y, yaxis4 = axis_y, yaxis5 = axis_y, yaxis6 = axis_y,
                     xaxis = axis_x, xaxis2 = axis_x, xaxis3 = axis_x, xaxis4 = axis_x, xaxis5 = axis_x, xaxis6 = axis_x,
                     )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=11)
    
    for element in fig.layout:
        if 'yaxis' in element and any([x in element[-1] for x in ['s','4']]):
            fig.layout[element]['title_text'] = title_y
            
        if 'xaxis' in element and any([x in element[-1] for x in ['4','5', '6']]):
            fig.layout[element]['title_text'] = 'Work You Enjoy, %'
        
    fig.show()
    
    return fig 

In [29]:
fig = wye_plot('PERFORMANCE_RATING', title_y='Performance Rate')

In [30]:
fig = wye_plot('SatisfiedWork', title_y='Satisfied Working')

In [31]:
fig = wye_plot('SkillsNeedCarreer', title_y='Skills Needed for Carreer')

In [32]:
fig = wye_plot('WorkManagerAgain', title_y='Work with Manager Again')

In [33]:
fig = wye_plot('YearsStay', title_y='Planning Years to Stay')

In [34]:
fig = wye_plot('LEVEL', title_y='Level')